In [ ]:
import os
import google.generativeai as genai


def count_parameters(file_path):
    """ 計算設定檔中的參數數量 """
    count = 0
    with open(file_path, "r") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line and not stripped_line.startswith("#"):
                count += 1
    return count

def split_oai_config(input_file):
    output_dir = "split_conf"
    os.makedirs(output_dir, exist_ok=True)
    
    sections = {
        "1_identification.conf": [],
        "2_physical_layer.conf": [],
        "3_mac_rlc.conf": [],
        "4_network.conf": [],
        "5_ru_l1.conf": [],
        "6_security_logging.conf": []
    }
    ordered_sections = []
    
    # 定義分類標籤
    identification_keys = ["Active_gNBs", "Asn1_verbosity", "gNB_ID", "gNB_name", "tracking_area_code", "plmn_list", "nr_cellid"]
    physical_layer_keys = ["pdsch_AntennaPorts_XP", "pusch_AntennaPorts", "do_CSIRS", "do_SRS", "sib1_tda", "servingCellConfigCommon"]
    mac_rlc_keys = ["MACRLCs"]
    network_keys = ["SCTP", "amf_ip_address", "NETWORK_INTERFACES"]
    ru_l1_keys = ["L1s", "RUs"]
    security_logging_keys = ["security", "log_config", "fhi_72"]

    current_section = None
    original_count = count_parameters(input_file)
    
    with open(input_file, "r") as file:
        for line in file:
            stripped_line = line.strip()
            
            if any(key in stripped_line for key in identification_keys):
                current_section = "1_identification.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in physical_layer_keys):
                current_section = "2_physical_layer.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in mac_rlc_keys):
                current_section = "3_mac_rlc.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in network_keys):
                current_section = "4_network.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in ru_l1_keys):
                current_section = "5_ru_l1.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in security_logging_keys):
                current_section = "6_security_logging.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            
            if current_section:
                sections[current_section].append(line)
    
    # 寫入各個分類的設定檔到 split_conf 資料夾
    for filename, content in sections.items():
        with open(os.path.join(output_dir, filename), "w") as out_file:
            out_file.writelines(content)
    
    # 確認參數數量一致
    split_count = sum(count_parameters(os.path.join(output_dir, file)) for file in sections.keys())
    if original_count == split_count:
        print("OAI 設定檔已成功拆分，且參數數量一致！")
        print("You can find the split configuration files in the 'split_conf' folder.")
    else:
        print(f"警告：參數數量不一致！原始: {original_count}, 分割後: {split_count}")

def merge_oai_config(output_file):
    """ 按照順序合併設定檔，確保格式正確 """
    output_dir = "split_conf"
    ordered_files = ["1_identification.conf", "2_physical_layer.conf", "3_mac_rlc.conf", "4_network.conf", "5_ru_l1.conf", "6_security_logging.conf"]
    
    with open(output_file, "w") as out_file:
        for file in ordered_files:
            file_path = os.path.join(output_dir, file)
            if os.path.exists(file_path):
                with open(file_path, "r") as in_file:
                    out_file.writelines(in_file.readlines())
                out_file.write("\n")
    print("OAI 設定檔已成功合併！")

def modify_parameter(user_request):
    """ 根據 Google AI API 解析結果修改設定參數 """
    output_dir = "split_conf"
    updated = False
    
    try:
        param_name, new_value = query_google_ai(user_request)
    except ValueError as e:
        print("錯誤:", e)
        return
    
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        with open(file_path, "r") as f:
            lines = f.readlines()
        
        with open(file_path, "w") as f:
            for line in lines:
                if param_name in line:
                    line = f'{param_name} = "{new_value}";\n'
                    updated = True
                f.write(line)
    
    if updated:
        print(f"成功修改 {param_name} 為 {new_value}！")
    else:
        print(f"警告：未找到 {param_name}，未進行修改！")

def query_google_ai(prompt):
    """ 呼叫 Google AI API，解析要修改的參數 """
    genai.configure(api_key="xx")
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    response = model.generate_content(prompt)
    
    response_text = response.text.strip()
    print("Google AI 回應:", response_text)  # Debugging log
    
    # 嘗試解析回傳的格式
    if "=" in response_text:
        param_data = response_text.split("=", 1)  # 只分割一次，避免多個 '=' 產生錯誤
        if len(param_data) == 2:
            param_name = param_data[0].strip()
            new_value = param_data[1].strip().replace(';', '').replace('"', '')
            return param_name, new_value
    
    raise ValueError("AI 回傳格式錯誤，無法解析參數")


# 執行切割、AI 解析修改與合併
split_oai_config("config_sample/sample.conf")
user_request = "請修改 GNB_IPV4_ADDRESS_FOR_NG_AMF = \"1.1.1.1\";"
modify_parameter(user_request)
merge_oai_config("final_modify_output/merged.conf")


OAI 設定檔已成功拆分，且參數數量一致！
You can find the split configuration files in the 'split_conf' folder.
Google AI 回應: This depends on what you want to change it *to*.  Here are a few options:

**1. Change to a different static IP address:**

```
GNB_IPV4_ADDRESS_FOR_NG_AMF = "10.0.0.1"; 
```

(Replace `10.0.0.1` with the desired IP address. Make sure it's a valid and accessible IP in your network.)

**2. Get the IP address dynamically (e.g., from the host machine's interface):**

This requires code, and the specific implementation depends on the programming language you're using. Here are examples in Python and bash:

**Python:**

```python
import socket
import netifaces

def get_ip_address(interface_name='eth0'): # Replace 'eth0' with the actual interface name
    try:
        addresses = netifaces.ifaddresses(interface_name)
        ipv4_addresses = addresses[netifaces.AF_INET]
        return ipv4_addresses[0]['addr']
    except (ValueError, KeyError): # Handle cases where the interface or IP i

# 有 UI

In [8]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox

def count_parameters(file_path):
    """ 計算設定檔中的參數數量 """
    count = 0
    with open(file_path, "r") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line and not stripped_line.startswith("#"):
                count += 1
    return count

def split_oai_config(input_file, label):
    output_dir = "split_conf"
    os.makedirs(output_dir, exist_ok=True)
    
    sections = {
        "1_identification.conf": [],
        "2_physical_layer.conf": [],
        "3_mac_rlc.conf": [],
        "4_network.conf": [],
        "5_ru_l1.conf": [],
        "6_security_logging.conf": []
    }
    
    identification_keys = ["Active_gNBs", "Asn1_verbosity", "gNB_ID", "gNB_name", "tracking_area_code", "plmn_list", "nr_cellid"]
    physical_layer_keys = ["pdsch_AntennaPorts_XP", "pusch_AntennaPorts", "do_CSIRS", "do_SRS", "sib1_tda", "servingCellConfigCommon"]
    mac_rlc_keys = ["MACRLCs"]
    network_keys = ["SCTP", "amf_ip_address", "NETWORK_INTERFACES"]
    ru_l1_keys = ["L1s", "RUs"]
    security_logging_keys = ["security", "log_config", "fhi_72"]

    current_section = None
    original_count = count_parameters(input_file)
    
    with open(input_file, "r") as file:
        for line in file:
            stripped_line = line.strip()
            
            if any(key in stripped_line for key in identification_keys):
                current_section = "1_identification.conf"
            elif any(key in stripped_line for key in physical_layer_keys):
                current_section = "2_physical_layer.conf"
            elif any(key in stripped_line for key in mac_rlc_keys):
                current_section = "3_mac_rlc.conf"
            elif any(key in stripped_line for key in network_keys):
                current_section = "4_network.conf"
            elif any(key in stripped_line for key in ru_l1_keys):
                current_section = "5_ru_l1.conf"
            elif any(key in stripped_line for key in security_logging_keys):
                current_section = "6_security_logging.conf"
            
            if current_section:
                sections[current_section].append(line)
    
    for filename, content in sections.items():
        with open(os.path.join(output_dir, filename), "w") as out_file:
            out_file.writelines(content)
    
    split_count = sum(count_parameters(os.path.join(output_dir, file)) for file in sections.keys())
    if original_count == split_count:
        messagebox.showinfo("成功", "OAI 設定檔已成功拆分，且參數數量一致！")
    else:
        messagebox.showwarning("警告", f"參數數量不一致！原始: {original_count}, 分割後: {split_count}")

def open_file(label):
    file_path = filedialog.askopenfilename(filetypes=[("Configuration Files", "*.conf")])
    if file_path:
        label.config(text=f"選擇的檔案: {file_path}")
        split_oai_config(file_path, label)

def create_gui():
    root = tk.Tk()
    root.title("OAI 設定檔管理工具")
    root.geometry("500x250")
    
    label = tk.Label(root, text="選擇要拆分的 OAI 設定檔:")
    label.pack(pady=10)
    
    file_label = tk.Label(root, text="未選擇檔案", fg="blue")
    file_label.pack(pady=10)
    
    button = tk.Button(root, text="選擇檔案", command=lambda: open_file(file_label))
    button.pack(pady=10)
    
    root.mainloop()

if __name__ == "__main__":
    create_gui()


# 無 UI

In [12]:
import os

def count_parameters(file_path):
    """ 計算設定檔中的參數數量 """
    count = 0
    with open(file_path, "r") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line and not stripped_line.startswith("#"):
                count += 1
    return count

def split_oai_config(input_file):
    output_dir = "split_conf"
    os.makedirs(output_dir, exist_ok=True)
    
    sections = {
        "1_identification.conf": [],
        "2_physical_layer.conf": [],
        "3_mac_rlc.conf": [],
        "4_network.conf": [],
        "5_ru_l1.conf": [],
        "6_security_logging.conf": []
    }
    ordered_sections = []
    
    # 定義分類標籤
    identification_keys = ["Active_gNBs", "Asn1_verbosity", "gNB_ID", "gNB_name", "tracking_area_code", "plmn_list", "nr_cellid"]
    physical_layer_keys = ["pdsch_AntennaPorts_XP", "pusch_AntennaPorts", "do_CSIRS", "do_SRS", "sib1_tda", "servingCellConfigCommon"]
    mac_rlc_keys = ["MACRLCs"]
    network_keys = ["SCTP", "amf_ip_address", "NETWORK_INTERFACES"]
    ru_l1_keys = ["L1s", "RUs"]
    security_logging_keys = ["security", "log_config", "fhi_72"]

    current_section = None
    original_count = count_parameters(input_file)
    
    with open(input_file, "r") as file:
        for line in file:
            stripped_line = line.strip()
            
            if any(key in stripped_line for key in identification_keys):
                current_section = "1_identification.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in physical_layer_keys):
                current_section = "2_physical_layer.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in mac_rlc_keys):
                current_section = "3_mac_rlc.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in network_keys):
                current_section = "4_network.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in ru_l1_keys):
                current_section = "5_ru_l1.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            elif any(key in stripped_line for key in security_logging_keys):
                current_section = "6_security_logging.conf"
                if current_section not in ordered_sections:
                    ordered_sections.append(current_section)
            
            if current_section:
                sections[current_section].append(line)
    
    # 寫入各個分類的設定檔到 split_conf 資料夾
    for filename, content in sections.items():
        with open(os.path.join(output_dir, filename), "w") as out_file:
            out_file.writelines(content)
    
    # 確認參數數量一致
    split_count = sum(count_parameters(os.path.join(output_dir, file)) for file in sections.keys())
    if original_count == split_count:
        print("OAI 設定檔已成功拆分，且參數數量一致！")
        print("You can find the split configuration files in the 'split_conf' folder.")
    else:
        print(f"警告：參數數量不一致！原始: {original_count}, 分割後: {split_count}")

def merge_oai_config(output_file):
    """ 按照順序合併設定檔，確保格式正確 """
    output_dir = "split_conf"
    ordered_files = ["1_identification.conf", "2_physical_layer.conf", "3_mac_rlc.conf", "4_network.conf", "5_ru_l1.conf", "6_security_logging.conf"]
    
    with open(output_file, "w") as out_file:
        for file in ordered_files:
            file_path = os.path.join(output_dir, file)
            if os.path.exists(file_path):
                with open(file_path, "r") as in_file:
                    out_file.writelines(in_file.readlines())
                out_file.write("\n")
    print("OAI 設定檔已成功合併！")

# 執行切割與合併功能
split_oai_config("config_sample\sample.conf")
merge_oai_config("final_modify_output\merged.conf")


OAI 設定檔已成功拆分，且參數數量一致！
You can find the split configuration files in the 'split_conf' folder.
OAI 設定檔已成功合併！
